In [6]:
import mediapipe as mp
import cv2
import numpy as np

In [7]:
mp_drawing  = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [8]:
cap = cv2.VideoCapture('ROWTRAIN.mp4')

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, image = cap.read()
        
        
        #recolor
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        #detections
        results = pose.process(image)
        
        #recolor back to RGB
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
                                 , mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                                 mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                 )
        
        
        
        
        
        cv2.imshow('RawFeed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
            
cap.release()
cv2.destroyAllWindows()

In [9]:
import csv
import os
import numpy as np

In [10]:
num_coords = len(results.pose_landmarks.landmark)

In [11]:
landmarks = ['class']
for val in range (1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [12]:
with open ('trainData.csv', mode='w', newline ='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [13]:
class_name = "Catch"

In [14]:
cap = cv2.VideoCapture('CatchTrain.mp4')

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, image = cap.read()
        
        
        #recolor
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        #detections
        results = pose.process(image)
        
        #recolor back to RGB
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
                                 , mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                                 mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                 )
        
        #export coordinates
        try:
            pose = results.pose_landmarks.landmark #gets the pose landmarks
            pose_row = np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten()
                #gives each landmark component
                # .flatten() collapses an array into a single dimension, [[1,2],[3,4]] --> [1,2,3,4]

            pose_row.insert(0, class_name) #appends the class name to the CSV


                #Exports to CSV

            with open ('trainData.csv', 'a', newline ='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(pose_row)
               
            
            
            
        except Exception as e:
            pass
        
        
        
        
        
        cv2.imshow('CatchFeed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
            
cap.release()
cv2.destroyAllWindows()

AttributeError: 'RepeatedCompositeFieldContainer' object has no attribute 'process'